<a href="https://colab.research.google.com/github/HonkinWaffles/Google-Colab-Notebooks/blob/main/Honkin_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion
Stable Diffusion (Official)
https://github.com/CompVis/stable-diffusion

Using Fork: https://github.com/basujindal/stable-diffusion

Model Data: v1.4


## Environment Setup

Setup Stable Diffusion

# Image Generation


### TXT2IMG

#### Setup

In [ ]:
%cd /content/
!pip install wget

import os, torch, gc, shutil, wget
from getpass import getpass
from pathlib import Path

#Login to GDRIVE for file access
LOGIN_GDRIVE = True
if LOGIN_GDRIVE and not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')



## Drive paths for Google Colab
# Root = /content/drive/MyDrive/GOOGLE_COLAB/
# Setup Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP
# img2img Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG
# txt2img Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG

## Make directory structure in drive to set everything up correctly
!mkdir -p /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG/INPUT /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG/OUTPUT /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG


## Check if file exists in Drive if not download - 

path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/Miniconda3-latest-Linux-x86_64.sh'
path = Path(path_to_file)

if path.is_file():
    shutil.copy2('/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/Miniconda3-latest-Linux-x86_64.sh', '/content/')
    print('Copying local version of Conda')
else:
    wget.download('https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh')
    print('Downloading Miniconda from internet - Consider adding a copy to `/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/`')


# Setup Miniconda
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda init bash
!conda install wget -y
!conda install diffusers -y 

# Clone the Basujindal Stable-Diffusion
!git clone https://github.com/basujindal/stable-diffusion.git
import os
os.chdir('stable-diffusion')
!mkdir -p models/ldm/stable-diffusion-v1/


## Copy over the model file from Drive
path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/sd-v1-4-full-ema.ckpt'
path = Path(path_to_file)

if path.is_file():
    print('Copying local version of models ')
    shutil.copy2('/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/sd-v1-4-full-ema.ckpt', 'models/ldm/stable-diffusion-v1/model.ckpt')
else:
    wget.download('https://www.googleapis.com/storage/v1/b/aai-blog-files/o/sd-v1-4.ckpt?alt=media','models/ldm/stable-diffusion-v1/model.ckpt' )
    print('Downloading model from internet - Consider adding the better copy from `Huggingface` to `/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/`')


!conda env update -n base -f environment.yaml

#### Generator

In [ ]:
#@markdown Fill out the prompt and image details then run the code to generate images inside of `drive/MyDrive/AI_PICS/TXT2IMG`

#@markdown ---
PROMPT = "Testing Prompt for specifc things" #@param {type:'string'}
STEPS = 75 #@param {type:"slider", min:5, max:500, step:5} 
SEED = 0 #@param {type:'integer'}
NUM_ITERS = 1 #@param {type:"slider", min:1, max:100, step:1} 
WIDTH = 576 #@param {type:"slider", min:256, max:1920, step:64}
HEIGHT = 704 #@param {type:"slider", min:256, max:1920, step:64}
SCALE = 15.3 #@param {type:"slider", min:0, max:25, step:0.1}
#@markdown ---

if SEED == 0:
  SEED=42

!python optimizedSD/optimized_txt2img.py \
--n_samples 1 \
--n_iter $NUM_ITERS \
--scale $SCALE \
--H $HEIGHT \
--W $WIDTH \
--seed $SEED \
--ddim_steps $STEPS \
--outdir /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG/ \
--prompt "$PROMPT"



### IMG2IMG


#### Setup

In [ ]:
%cd /content/
!pip install wget

import os, torch, gc, shutil, wget
from getpass import getpass
from pathlib import Path

#Login to GDRIVE for file access
LOGIN_GDRIVE = True
if LOGIN_GDRIVE and not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')



## Drive paths for Google Colab
# Root = /content/drive/MyDrive/GOOGLE_COLAB/
# Setup Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP
# img2img Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG
# txt2img Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG

## Make directory structure in drive to set everything up correctly
!mkdir -p /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG/INPUT /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG/OUTPUT /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG


## Check if file exists in Drive if not download - 

path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/Miniconda3-latest-Linux-x86_64.sh'
path = Path(path_to_file)

if path.is_file():
    shutil.copy2('/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/Miniconda3-latest-Linux-x86_64.sh', '/content/')
    print('Copying local version of Conda')
else:
    wget.download('https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh')
    print('Downloading Miniconda from internet - Consider adding a copy to `/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/`')


# Setup Miniconda
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda init bash
!conda install wget -y
!conda install diffusers -y 

# Clone the Basujindal Stable-Diffusion
!git clone https://github.com/basujindal/stable-diffusion.git
import os
os.chdir('stable-diffusion')
!mkdir -p models/ldm/stable-diffusion-v1/


## Copy over the model file from Drive
path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/sd-v1-4-full-ema.ckpt'
path = Path(path_to_file)

if path.is_file():
    print('Copying local version of models ')
    shutil.copy2('/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/sd-v1-4-full-ema.ckpt', 'models/ldm/stable-diffusion-v1/model.ckpt')
else:
    wget.download('https://www.googleapis.com/storage/v1/b/aai-blog-files/o/sd-v1-4.ckpt?alt=media','models/ldm/stable-diffusion-v1/model.ckpt' )
    print('Downloading model from internet - Consider adding the better copy from `Huggingface` to `/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/`')


!conda env update -n base -f environment.yaml

#### Generator

In [ ]:
import os
from google.colab import files
import shutil

#@markdown Modifies and existing images using a prompt generated by the user


#@markdown ---

PROMPT = "Big car go fast" #@param {type:'string'}
STEPS = 100 #@param {type:"slider", min:5, max:500, step:5} 
NUM_ITERS = 1 #@param {type:"slider", min:1, max:100, step:1} 
#@markdown ---

!mkdir -p upload
upload_folder = 'upload'

# upload images
uploaded = files.upload()

for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)


!python optimizedSD/optimized_img2img.py \
--prompt "$PROMPT" \
--n_iter $NUM_ITERS \
--strength 0.7 \
--ddim_steps $STEPS \
--outdir /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG/OUTPUT/ \
--init-img $dst_path


## Bulk IMG2IMG

### Setup

In [3]:

# Drive paths for Google Colab
# Root = /content/drive/MyDrive/GOOGLE_COLAB/
# Setup Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP
# img2img Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/IMG2IMG
# txt2img Files = /content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG


# Install Requirements 
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"
!pip install wget

# Load Requirements
import os, torch, gc, shutil, wget
from getpass import getpass
from pathlib import Path
from huggingface_hub import notebook_login
from google.colab import output

#Login to GDRIVE for file access
LOGIN_GDRIVE = True
if LOGIN_GDRIVE and not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

# Login to Hugging Face API
path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/SETUP/token'
path = Path(path_to_file)

if path.is_file():
    text_file = open(path_to_file, "r")
    token = text_file.read()
    text_file.close()
    !echo $token | huggingface-cli login
else:
    print('No token found please login manually')
    output.enable_custom_widget_manager()
    notebook_login()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 5.1 MB/s 
     |████████████████████████████████| 120 kB 30.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=051c3a71e6b50a0e19d15a2e68611df8a3838a2c54e317059709b90fd8b15126
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfull

### Start Pipeline

In [5]:
import torch
import os
import shutil 
import subprocess
from diffusers import StableDiffusionPipeline
from torch import autocast
from traitlets.config.loader import DeferredConfigString
from google.colab import files

# make sure you're logged in with `huggingface-cli login`
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True)  
pipe = pipe.to("cuda")

def dummy(images, **kwargs):
    return images, False
pipe.safety_checker = dummy

### Run Bulk Prompts

In [ ]:
import torch
import os
import shutil 
import subprocess
from diffusers import StableDiffusionPipeline
from torch import autocast
from traitlets.config.loader import DeferredConfigString
from google.colab import files

#@markdown Fill out the prompt and image details then run the code to generate images inside of `drive/MyDrive/AI_PICS/TXT2IMG`
#@markdown ---
STEPS=120 #@param {type:"slider", min:5, max:500, step:5} 
SEED=0 #@param {type:'integer'}
NUM_ITERS=1 #@param {type:"slider", min:1, max:100, step:1} 
WIDTH=512 #@param {type:"slider", min:256, max:1920, step:64}
HEIGHT=512 #@param {type:"slider", min:256, max:1920, step:64}
SCALE=15.3 #@param {type:"slider", min:0, max:25, step:0.1}
OUTDIR="22-09-03-TEST-LONG" #@param {type:'string'}
#@markdown ---
OUTPUT="/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG/" + OUTDIR + "/"

pipe.safety_checker = lambda images, **kwargs: (images, False)


upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  shutil.move(filename, dst_path)

prompts_file = open(dst_path)
prompts_imported = prompts_file.read()
prompts_split = prompts_imported.splitlines()

for x in range(NUM_ITERS):
  for i in prompts_split:
    with autocast("cuda"):
      image = pipe(i, num_inference_steps=STEPS, height=HEIGHT, width=WIDTH)["sample"][0]
      as_string = str(x)
      path = OUTPUT+i+'/'
      isExist = os.path.exists(path)
      if not isExist:
        os.makedirs(path)
        print("The new directory is created!")
      image.save(OUTPUT+i+'/'+i+as_string+'.png', 'PNG')




Saving promts.txt to promts.txt


0it [00:00, ?it/s]